In [ ]:
import pickle
import pandas as pd

from sklearn.model_selection import train_test_split
import numpy as np

# (SYNTHETIC DATA) Benchmarking vs. other paired models

## Data

In [ ]:
comparisons = pickle.load(open('../data/comparisons_df.pickle', 'rb'))

In [ ]:
seed = 30

In [ ]:
comparisons = comparisons.copy()
comparisons['image_l'] = comparisons['scene_i']
comparisons['image_r'] = comparisons['scene_j']
comparisons['Winner'] = -1
comparisons['Loser'] = -1
comparisons['Tie'] = 0

for i, row in comparisons.iterrows():
    l_item = row.image_l
    r_item = row.image_r
    
    if row.score == 1:
        comparisons.loc[i, 'Winner'] = r_item
        comparisons.loc[i, 'Loser'] = l_item
        comparisons.loc[i, 'Tie'] = 0
    elif row.score == -1:
        comparisons.loc[i, 'Winner'] = l_item
        comparisons.loc[i, 'Loser'] = r_item
        comparisons.loc[i, 'Tie'] = 0
    elif row.score == 0:
        comparisons.loc[i, 'Winner'] = r_item
        comparisons.loc[i, 'Loser'] = l_item
        comparisons.loc[i, 'Tie'] = 1


In [ ]:
unique_images = pd.unique(comparisons[['image_l', 'image_r']].values.ravel('K'))

Split data in Train, Validation & Test

In [ ]:
X_train, X_test = train_test_split(comparisons, test_size=0.2, random_state=seed)
X_train, X_val  = train_test_split(X_train, test_size=0.13, random_state=seed)

## TrueSkill

In [ ]:
import itertools
import math
import trueskill as ts

In [ ]:
def compute_probabilities(team1, team2):
    BETA = ts.BETA
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
    ts_ = ts.global_env()
    return ts_.cdf(delta_mu / denom), 1 - ts_.cdf(delta_mu / denom)

In [ ]:
def compute_accuracy(df):
    accuracy = []
    for i, row in df.iterrows():
        p_win, p_los = compute_probabilities([scores[row.Winner]], [scores[row.Loser]])
    
        if row.score == -1 or row.score == 1:
            accuracy.append(int(p_win > p_los))
    
    return accuracy

### Run

In [ ]:
# Initialize TrueSkill scores
scores = {}

for image in unique_images:
    scores[image] = ts.Rating()

# Compute scores based on comparisons
for i, row in X_train.iterrows():
    # Define the players in this round
    player1 = scores[row['image_l']]
    player2 = scores[row['image_r']]
    
    # Process match
    if row['score'] == -1:
        score = [0, 1]
    elif row['score'] == 0:
        score = [0, 0]
    elif row['score'] == 1:
        score = [1, 0]
    
    [player1], [player2] = ts.rate([[player1], [player2]], ranks=score)

    # Update scores
    scores[row['image_l']] = player1
    scores[row['image_r']] = player2

In [ ]:
accuracy_train = compute_accuracy(X_train[X_train.score != 0])
accuracy_test = compute_accuracy(X_test[X_test.score != 0])

# Compile results
result_ts = {
    'model': 'trueskill',
    'train_accuracy': np.mean(accuracy_train),
    'test_accuracy': np.mean(accuracy_test),
    'seed': seed
}

In [ ]:
result_ts

## Elo

In [ ]:
class OriginalELo(object):
    def __init__(self, k_factor, elo_width, starting_elo):
        self.k_factor = k_factor
        self.elo_width = elo_width
        self.starting_elo = starting_elo
        self.items = set()
        self.items_elo = dict()
        
    def initialize_items(self, items):
        """Initialize the items available to `items`."""
        self.items = set(items)
    
    def initialize_elos(self, ):
        """Set the initial starting elo for all available items."""
        for item in self.items:
            self.items_elo[item] = self.starting_elo
    
    def expected_result(self, elo_a, elo_b):
        """Expected probability of item with elo_a winning vs. item with elo_b."""
        
        expect_a = 1.0/(1+10**((elo_b - elo_a)/self.elo_width))
        return expect_a
    
    def update_elo(self, winner_elo, loser_elo, tie=False):
        """Update elo for the winning item and losing item."""
        
        R = 1
        if tie:
            R = .5
        
        expected_win = self.expected_result(winner_elo, loser_elo)  
        change_in_elo = self.k_factor * (R-expected_win)
        
        winner_elo += change_in_elo
        loser_elo -= change_in_elo
        return winner_elo, loser_elo
    
    def add_comparison(self, w_item, l_item, tie=False):
        """Process comparison between winning item and losing item."""
        current_winner_elo = self.items_elo[w_item]
        current_loser_elo = self.items_elo[l_item]
        
        updated_winner_elo, updated_loser_elo = self.update_elo(current_winner_elo, current_loser_elo, tie=tie)
        
        self.items_elo[w_item] = updated_winner_elo
        self.items_elo[l_item] = updated_loser_elo
        

In [ ]:
def compute_probabilities(elo_a, elo_b, allow_ties=False):
    """
    Expected probabilities of winning, drawing, or losing.
    Reference for draws formula: `Mathematical Model of Ranking Accuracy and Popularity Promotion`
    https://www.researchgate.net/publication/309662241_Mathematical_Model_of_Ranking_Accuracy_and_Popularity_Promotion
    """
      
    p_win = 1. / (1+10**((-elo_a + elo_b)/elo_width))
    p_los = 1. / (1+10**((elo_a - elo_b)/elo_width))
    
    if allow_ties:
        p_tie = (1 / (np.sqrt(2 * np.pi) * np.e)) * np.exp(-1 * (( (elo_a-elo_b)/(elo_width/2) )**2) / (2*np.e**2))
        p_win = p_win - 0.5 * p_tie  
        p_los = p_los - 0.5 * p_tie
        
        return  p_win, p_los, p_tie

    return p_win, p_los
    
def compute_logloss(df):
    log_loss = []
    for i, row in df.iterrows():
        # p_win, p_los, p_tie = compute_probabilities(elo.items_elo[row.Winner], elo.items_elo[row.Loser])
        p_win, p_los = compute_probabilities(elo.items_elo[row.Winner], elo.items_elo[row.Loser])

        if row.score == -1 or row.score == 1:
            log_loss.append(np.log(p_win))
        else:
            log_loss.append(np.log(p_tie))
    
    return log_loss
    
def compute_accuracy(df):
    accuracy = []
    
    for i, row in df.iterrows():
        # p_win, p_los, p_tie = compute_probabilities(elo.items_elo[row.Winner], elo.items_elo[row.Loser])
        p_win, p_los = compute_probabilities(elo.items_elo[row.Winner], elo.items_elo[row.Loser])

        if row.score == -1 or row.score == 1:
            accuracy.append(int(p_win > p_los))
        else:
            accuracy.append(int(p_win > p_los))
    
    return accuracy

In [ ]:
# ELO
starting_elo = 1500
elo_width = 400
k_factor = 32

elo = OriginalELo(k_factor=k_factor, 
                  elo_width=elo_width, 
                  starting_elo=starting_elo)
elo.initialize_items(list(comparisons.Winner.values) + list(comparisons.Loser.values))
elo.initialize_elos()

### Run

In [ ]:
for i, row in X_train.iterrows():
    w_item = row.Winner
    l_item = row.Loser 
    tie = True if row.Tie else False
    
    elo.add_comparison(w_item, l_item, tie=tie)    
scores = []

for item, item_elo in elo.items_elo.items():
    scores.append({
        'score': item_elo,
        'image': item,
    })
scores_df = pd.DataFrame(scores).set_index('image', drop=False)

accuracy_train = compute_accuracy(X_train[X_train.score != 0])
accuracy_test = compute_accuracy(X_test[X_test.score != 0])

# Compile results
result_elo = {
    'model': 'elo',
    'train_accuracy': np.mean(accuracy_train),
    'test_accuracy': np.mean(accuracy_test),
    'seed': seed
}

In [ ]:
result_elo

## Gaussian Process

In [ ]:
import kickscore as ks

In [ ]:
def compute_accuracy(df, t_):
    accuracy = []
    
    for i, row in df.iterrows():
        try:
            p_win, p_tie, p_los = model.probabilities([row.Winner], [row.Loser], t=t_)

            if row.score == -1 or row.score == 1:
                if p_win > p_los and p_win > p_tie:
                    accuracy.append(1) 
                else:
                    accuracy.append(0) 
            else:  
                if p_tie > p_los and p_tie > p_win:
                    accuracy.append(1) 
                else:
                    accuracy.append(0) 
        except KeyError:
            continue
    return accuracy

### Run

In [ ]:
images = set()
observations = list()

for i, row in X_train.reset_index().iterrows():
    t = i
    images.add(row.image_l)
    images.add(row.image_r)
    
    if row.score == -1:
        observations.append({
                'winners': [row.image_l],
                'losers': [row.image_r],
                #'tie': False,
                't': t,
            })
    if row.score == 0:
        observations.append({
                'winners': [row.image_l],
                'losers': [row.image_r],
                'tie': True,
                't': t,
            })
    if row.score == 1:
        observations.append({
                'winners': [row.image_r],
                'losers': [row.image_l],
                #'tie': False,
                't': t,
            })

model = ks.TernaryModel(margin=0.2)
kernel = (ks.kernel.Constant(var=0.03))
for image in images:
    model.add_item(image, kernel=kernel)
for obs in observations:
    model.observe(**obs)

converged = model.fit()
if converged:
    print("Model has converged.")

ts = [comparisons.shape[0] ]  # Point in time at which you want to make the prediction.
res = dict()  # Contains predicted score.

scores = []
for name, item in model.item.items():
    means, var = item.predict(ts)
    scores += [[name, means[0], var[0]]]

scores = pd.DataFrame(scores, columns=['image', 'score', 'var'])
scores_df = scores.sort_values(by='image').reset_index(drop=True)

scores_df = scores_df.set_index('image', drop=False)
scores_df.index.name = None

accuracy_train = compute_accuracy(X_train[X_train.score != 0], comparisons.shape[0])
accuracy_test = compute_accuracy(X_test[X_test.score != 0], comparisons.shape[0])

# Compile results
result_gp = {
    'model': 'gaussian_process',
    'train_accuracy': np.mean(accuracy_train),
    'test_accuracy': np.mean(accuracy_test),
    'seed': seed
}

In [ ]:
result_gp

## Rank Centrality

In [ ]:
import choix

In [ ]:
def compute_accuracy(df, params):
    accuracy = []
    
    for i, row in df.iterrows():
        if row.score == -1:
            p_win, p_los = choix.probabilities([int(row.image_l), int(row.image_r)], params)
        elif row.score == 1:
            p_win, p_los = choix.probabilities([int(row.image_r), int(row.image_l)], params)

        if row.score == -1 or row.score == 1:
            if p_win > p_los:
                accuracy.append(1) 
            else:
                accuracy.append(0)  

    return accuracy

### Run

In [ ]:
comparisons_ = comparisons.copy()
# RANK CENTRALITY
n_items = len(pd.unique(comparisons_[['image_l', 'image_r']].values.ravel('K')))
images = pd.unique(comparisons_[['image_l', 'image_r']].values.ravel('K'))
images_dict = {}
images_dict_rev = {}
for i, image_id in enumerate(images):
    images_dict[image_id] = i
    images_dict_rev[i] = image_id
comparisons_=comparisons_.replace({"image_l": images_dict})
comparisons_=comparisons_.replace({"image_r": images_dict})
comparisons_=comparisons_.replace({"Winner": images_dict})
comparisons_=comparisons_.replace({"Loser": images_dict})

# Split data in Train, Validation & Test
X_train, X_test = train_test_split(comparisons_, test_size=0.2, random_state=seed)
X_train, X_val  = train_test_split(X_train, test_size=0.13, random_state=seed)

data = []
for i, row in X_train.iterrows():
    if not row.Tie:
        data.append((int(row.Winner), int(row.Loser)))

    if row.Tie:
        data.append((row.Winner, row.Loser))
        data.append((row.Loser, row.Winner))

params_rc = choix.rank_centrality(n_items, data, alpha=1e-4)

accuracy_train = compute_accuracy(X_train[X_train.score != 0], params_rc)
accuracy_test = compute_accuracy(X_test[X_test.score != 0], params_rc)

# Compile results
result_rc = {
    'model': 'rank_centrality',
    'train_accuracy': np.mean(accuracy_train),
    'test_accuracy': np.mean(accuracy_test),
    'seed': seed
}

In [ ]:
result_rc

# (REAL DATA) Benchmarking vs. other paired models

## Data

In [ ]:
comparisons = pickle.load(open('../data/berlin_comparisons_df.pickle', 'rb'))

In [ ]:
seed = 30

In [ ]:
comparisons = comparisons.copy()
comparisons['Winner'] = -1
comparisons['Loser'] = -1
comparisons['Tie'] = 0

for i, row in comparisons.iterrows():
    l_item = row.image_l
    r_item = row.image_r
    
    if row.score == 1:
        comparisons.loc[i, 'Winner'] = r_item
        comparisons.loc[i, 'Loser'] = l_item
        comparisons.loc[i, 'Tie'] = 0
    elif row.score == -1:
        comparisons.loc[i, 'Winner'] = l_item
        comparisons.loc[i, 'Loser'] = r_item
        comparisons.loc[i, 'Tie'] = 0
    elif row.score == 0:
        comparisons.loc[i, 'Winner'] = r_item
        comparisons.loc[i, 'Loser'] = l_item
        comparisons.loc[i, 'Tie'] = 1


In [ ]:
unique_images = pd.unique(comparisons[['image_l', 'image_r']].values.ravel('K'))

Split data in Train, Validation & Test

In [ ]:
X_train, X_test = train_test_split(comparisons, test_size=0.2, random_state=seed)
X_train, X_val  = train_test_split(X_train, test_size=0.13, random_state=seed)

## TrueSkill

In [ ]:
import itertools
import math
import trueskill as ts

In [ ]:
def compute_probabilities(team1, team2):
    BETA = ts.BETA
    delta_mu = sum(r.mu for r in team1) - sum(r.mu for r in team2)
    sum_sigma = sum(r.sigma ** 2 for r in itertools.chain(team1, team2))
    size = len(team1) + len(team2)
    denom = math.sqrt(size * (BETA * BETA) + sum_sigma)
    ts_ = ts.global_env()
    return ts_.cdf(delta_mu / denom), 1 - ts_.cdf(delta_mu / denom)

In [ ]:
def compute_accuracy(df):
    accuracy = []
    for i, row in df.iterrows():
        p_win, p_los = compute_probabilities([scores[row.Winner]], [scores[row.Loser]])
    
        if row.score == -1 or row.score == 1:
            accuracy.append(int(p_win > p_los))
    
    return accuracy

### Run

In [ ]:
# Initialize TrueSkill scores
scores = {}

for image in unique_images:
    scores[image] = ts.Rating()

# Compute scores based on comparisons
for i, row in X_train.iterrows():
    # Define the players in this round
    player1 = scores[row['image_l']]
    player2 = scores[row['image_r']]
    
    # Process match
    if row['score'] == -1:
        score = [0, 1]
    elif row['score'] == 0:
        score = [0, 0]
    elif row['score'] == 1:
        score = [1, 0]
    
    [player1], [player2] = ts.rate([[player1], [player2]], ranks=score)

    # Update scores
    scores[row['image_l']] = player1
    scores[row['image_r']] = player2

In [ ]:
accuracy_train = compute_accuracy(X_train[X_train.score != 0])
accuracy_test = compute_accuracy(X_test[X_test.score != 0])

# Compile results
result_ts = {
    'model': 'trueskill',
    'train_accuracy': np.mean(accuracy_train),
    'test_accuracy': np.mean(accuracy_test),
    'seed': seed
}

In [ ]:
result_ts

## Elo

In [ ]:
class OriginalELo(object):
    def __init__(self, k_factor, elo_width, starting_elo):
        self.k_factor = k_factor
        self.elo_width = elo_width
        self.starting_elo = starting_elo
        self.items = set()
        self.items_elo = dict()
        
    def initialize_items(self, items):
        """Initialize the items available to `items`."""
        self.items = set(items)
    
    def initialize_elos(self, ):
        """Set the initial starting elo for all available items."""
        for item in self.items:
            self.items_elo[item] = self.starting_elo
    
    def expected_result(self, elo_a, elo_b):
        """Expected probability of item with elo_a winning vs. item with elo_b."""
        
        expect_a = 1.0/(1+10**((elo_b - elo_a)/self.elo_width))
        return expect_a
    
    def update_elo(self, winner_elo, loser_elo, tie=False):
        """Update elo for the winning item and losing item."""
        
        R = 1
        if tie:
            R = .5
        
        expected_win = self.expected_result(winner_elo, loser_elo)  
        change_in_elo = self.k_factor * (R-expected_win)
        
        winner_elo += change_in_elo
        loser_elo -= change_in_elo
        return winner_elo, loser_elo
    
    def add_comparison(self, w_item, l_item, tie=False):
        """Process comparison between winning item and losing item."""
        current_winner_elo = self.items_elo[w_item]
        current_loser_elo = self.items_elo[l_item]
        
        updated_winner_elo, updated_loser_elo = self.update_elo(current_winner_elo, current_loser_elo, tie=tie)
        
        self.items_elo[w_item] = updated_winner_elo
        self.items_elo[l_item] = updated_loser_elo
        

In [ ]:
def compute_probabilities(elo_a, elo_b, allow_ties=False):
    """
    Expected probabilities of winning, drawing, or losing.
    Reference for draws formula: `Mathematical Model of Ranking Accuracy and Popularity Promotion`
    https://www.researchgate.net/publication/309662241_Mathematical_Model_of_Ranking_Accuracy_and_Popularity_Promotion
    """
      
    p_win = 1. / (1+10**((-elo_a + elo_b)/elo_width))
    p_los = 1. / (1+10**((elo_a - elo_b)/elo_width))
    
    if allow_ties:
        p_tie = (1 / (np.sqrt(2 * np.pi) * np.e)) * np.exp(-1 * (( (elo_a-elo_b)/(elo_width/2) )**2) / (2*np.e**2))
        p_win = p_win - 0.5 * p_tie  
        p_los = p_los - 0.5 * p_tie
        
        return  p_win, p_los, p_tie

    return p_win, p_los
    
def compute_logloss(df):
    log_loss = []
    for i, row in df.iterrows():
        # p_win, p_los, p_tie = compute_probabilities(elo.items_elo[row.Winner], elo.items_elo[row.Loser])
        p_win, p_los = compute_probabilities(elo.items_elo[row.Winner], elo.items_elo[row.Loser])

        if row.score == -1 or row.score == 1:
            log_loss.append(np.log(p_win))
        else:
            log_loss.append(np.log(p_tie))
    
    return log_loss
    
def compute_accuracy(df):
    accuracy = []
    
    for i, row in df.iterrows():
        # p_win, p_los, p_tie = compute_probabilities(elo.items_elo[row.Winner], elo.items_elo[row.Loser])
        p_win, p_los = compute_probabilities(elo.items_elo[row.Winner], elo.items_elo[row.Loser])

        if row.score == -1 or row.score == 1:
            accuracy.append(int(p_win > p_los))
        else:
            accuracy.append(int(p_win > p_los))
    
    return accuracy

In [ ]:
# ELO
starting_elo = 1500
elo_width = 400
k_factor = 32

elo = OriginalELo(k_factor=k_factor, 
                  elo_width=elo_width, 
                  starting_elo=starting_elo)
elo.initialize_items(list(comparisons.Winner.values) + list(comparisons.Loser.values))
elo.initialize_elos()

### Run

In [ ]:
for i, row in X_train.iterrows():
    w_item = row.Winner
    l_item = row.Loser 
    tie = True if row.Tie else False
    
    elo.add_comparison(w_item, l_item, tie=tie)    
scores = []

for item, item_elo in elo.items_elo.items():
    scores.append({
        'score': item_elo,
        'image': item,
    })
scores_df = pd.DataFrame(scores).set_index('image', drop=False)

accuracy_train = compute_accuracy(X_train[X_train.score != 0])
accuracy_test = compute_accuracy(X_test[X_test.score != 0])

# Compile results
result_elo = {
    'model': 'elo',
    'train_accuracy': np.mean(accuracy_train),
    'test_accuracy': np.mean(accuracy_test),
    'seed': seed
}

In [ ]:
result_elo

## Gaussian Process

In [ ]:
import kickscore as ks

In [ ]:
def compute_accuracy(df, t_):
    accuracy = []
    
    for i, row in df.iterrows():
        try:
            p_win, p_tie, p_los = model.probabilities([row.Winner], [row.Loser], t=t_)

            if row.score == -1 or row.score == 1:
                if p_win > p_los and p_win > p_tie:
                    accuracy.append(1) 
                else:
                    accuracy.append(0) 
            else:  
                if p_tie > p_los and p_tie > p_win:
                    accuracy.append(1) 
                else:
                    accuracy.append(0) 
        except KeyError:
            continue
    return accuracy

### Run

In [ ]:
images = set()
observations = list()

for i, row in X_train.reset_index().iterrows():
    t = i
    images.add(row.image_l)
    images.add(row.image_r)
    
    if row.score == -1:
        observations.append({
                'winners': [row.image_l],
                'losers': [row.image_r],
                #'tie': False,
                't': t,
            })
    if row.score == 0:
        observations.append({
                'winners': [row.image_l],
                'losers': [row.image_r],
                'tie': True,
                't': t,
            })
    if row.score == 1:
        observations.append({
                'winners': [row.image_r],
                'losers': [row.image_l],
                #'tie': False,
                't': t,
            })

model = ks.TernaryModel(margin=0.2)
kernel = (ks.kernel.Constant(var=0.03))
for image in images:
    model.add_item(image, kernel=kernel)
for obs in observations:
    model.observe(**obs)

converged = model.fit()
if converged:
    print("Model has converged.")

ts = [comparisons.shape[0] ]  # Point in time at which you want to make the prediction.
res = dict()  # Contains predicted score.

scores = []
for name, item in model.item.items():
    means, var = item.predict(ts)
    scores += [[name, means[0], var[0]]]

scores = pd.DataFrame(scores, columns=['image', 'score', 'var'])
scores_df = scores.sort_values(by='image').reset_index(drop=True)

scores_df = scores_df.set_index('image', drop=False)
scores_df.index.name = None

accuracy_train = compute_accuracy(X_train[X_train.score != 0], comparisons.shape[0])
accuracy_test = compute_accuracy(X_test[X_test.score != 0], comparisons.shape[0])

# Compile results
result_gp = {
    'model': 'gaussian_process',
    'train_accuracy': np.mean(accuracy_train),
    'test_accuracy': np.mean(accuracy_test),
    'seed': seed
}

In [ ]:
result_gp

## Rank Centrality

In [ ]:
import choix

In [ ]:
def compute_accuracy(df, params):
    accuracy = []
    
    for i, row in df.iterrows():
        if row.score == -1:
            p_win, p_los = choix.probabilities([int(row.image_l), int(row.image_r)], params)
        elif row.score == 1:
            p_win, p_los = choix.probabilities([int(row.image_r), int(row.image_l)], params)

        if row.score == -1 or row.score == 1:
            if p_win > p_los:
                accuracy.append(1) 
            else:
                accuracy.append(0)  

    return accuracy

### Run

In [ ]:
comparisons_ = comparisons.copy()
# RANK CENTRALITY
n_items = len(pd.unique(comparisons_[['image_l', 'image_r']].values.ravel('K')))
images = pd.unique(comparisons_[['image_l', 'image_r']].values.ravel('K'))
images_dict = {}
images_dict_rev = {}
for i, image_id in enumerate(images):
    images_dict[image_id] = i
    images_dict_rev[i] = image_id
comparisons_=comparisons_.replace({"image_l": images_dict})
comparisons_=comparisons_.replace({"image_r": images_dict})
comparisons_=comparisons_.replace({"Winner": images_dict})
comparisons_=comparisons_.replace({"Loser": images_dict})

# Split data in Train, Validation & Test
X_train, X_test = train_test_split(comparisons_, test_size=0.2, random_state=seed)
X_train, X_val  = train_test_split(X_train, test_size=0.13, random_state=seed)

data = []
for i, row in X_train.iterrows():
    if not row.Tie:
        data.append((int(row.Winner), int(row.Loser)))

    if row.Tie:
        data.append((row.Winner, row.Loser))
        data.append((row.Loser, row.Winner))

params_rc = choix.rank_centrality(n_items, data, alpha=1e-4)

accuracy_train = compute_accuracy(X_train[X_train.score != 0], params_rc)
accuracy_test = compute_accuracy(X_test[X_test.score != 0], params_rc)

# Compile results
result_rc = {
    'model': 'rank_centrality',
    'train_accuracy': np.mean(accuracy_train),
    'test_accuracy': np.mean(accuracy_test),
    'seed': seed
}

In [ ]:
result_rc